### HW Problem 2

In [2]:
# Imports
import numpy as np
import tensorflow as tf

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import numpy as np
import time

import matplotlib.pyplot as plt
tf.logging.set_verbosity(tf.logging.INFO)


### Configuration setup

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33

### Convert into spectograms

In [4]:
import librosa
s, sr=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)

sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)

In [5]:
mag_S=np.abs(S)
mag_X=np.abs(X)
# print(mag_S.shape, mag_X.shape, mag_Stest.shape)

### Create the model

In [35]:
def xavierfunc(shape):
    return np.sqrt(2.0/sum(shape))

In [36]:

x = tf.placeholder(tf.float32, [None, 513])
y_ = tf.placeholder(tf.float32, [None, 513]) # original

# W1 = tf.Variable(tf.truncated_normal([513, 1024], stddev = np.sqrt(1.0/sum([513, 1024])) ))
# b1 = tf.Variable(tf.truncated_normal([1024], stddev = np.sqrt(1.0/sum([1024])) ))

W1 = tf.Variable(tf.truncated_normal([513, 1024], stddev = xavierfunc([513, 1024]) ))
b1 = tf.Variable(tf.truncated_normal([1024], stddev = xavierfunc([1024]) ))

W2 = tf.Variable(tf.truncated_normal([1024, 1024], stddev = xavierfunc([1024, 1024]) ))
b2 = tf.Variable(tf.truncated_normal([1024], stddev = xavierfunc([1024]) ))

W3 = tf.Variable(tf.truncated_normal([1024, 513], stddev = xavierfunc([1024, 513]) ))
b3 = tf.Variable(tf.truncated_normal([513], stddev = xavierfunc([513]) ))

y1 = tf.nn.relu((x@W1) + b1)
y2 = tf.nn.relu((y1@W2) + b2)
y_pred = tf.nn.relu((y2@W3) + b3 )# predicted

mean_sqr = tf.losses.mean_squared_error(labels=y_, predictions=y_pred)
train_model = tf.train.AdamOptimizer().minimize(mean_sqr)

In [38]:
sess = tf.InteractiveSession(config=config)
tf.global_variables_initializer().run()


### Run the model

In [39]:
# Train
for i in range(1000):
    ts = sess.run(train_model, feed_dict={x: mag_X.T, y_: mag_S.T})


### Test the model

In [40]:
s, sr=librosa.load('test_x_01.wav', sr=None)
Stest_01=librosa.stft(s, n_fft=1024, hop_length=512)
mag_Stest_01=np.abs(Stest_01)

In [41]:
# Test trained model
Stest_hat_01=sess.run(y_pred, feed_dict={x: mag_Stest_01.T})
S_hat_01=(Stest_01/mag_Stest_01)*Stest_hat_01.T

S_time_01=librosa.istft(S_hat_01, hop_length=512)
librosa.output.write_wav('test_s_01_recons.wav', S_time_01, sr)

In [42]:
s, sr=librosa.load('test_x_02.wav', sr=None)
Stest_02=librosa.stft(s, n_fft=1024, hop_length=512)
mag_Stest_02=np.abs(Stest_02)

In [43]:
# Test trained model
Stest_hat_02=sess.run(y_pred, feed_dict={x: mag_Stest_02.T})
S_hat_02=(Stest_02/mag_Stest_02)*Stest_hat_02.T

S_time_02=librosa.istft(S_hat_02, hop_length=512)
librosa.output.write_wav('test_s_02_recons.wav', S_time_02, sr)